<center> <h1> Aircraft Data Response Extraction and Dataframe Compilation </h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><h4>Constructor Academy 2024</h4></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> Extract data from aircraft response API and match it to Squawk7700 data to build a unified dataframe</p>
        <strong>Keywords:</strong> data cleaning, data compiling
    </div>
</div>
</nav>

<a id='SU' name="SU"></a>
## [Set up](#P0)

### Packages

In [164]:
# General
import pandas as pd
import numpy as np
import json

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt


### Custom classes and functions

In [165]:
# NA

### Global Parameters Setting

In [166]:
# plt.style.use("seaborn")

### User-Dependent Variables

In [167]:
# NA

<a id='P1'></a>
## [Extract Meaningful Data from JSON Response File](#P0)

Extract aircraft response data and visualize one entry to decide what information is useful

In [168]:
with open('../data/raw/holy_grail_registration_responses.json', 'r') as reg_file:
    ac_response_data_reg = json.load(reg_file)

with open('../data/raw/holy_grail_icao24_responses.json', 'r') as icao_file:
    ac_response_data_icao = json.load(icao_file)


In [169]:
pretty_ac_json_reg = json.dumps(ac_response_data_reg[0], indent=4, sort_keys=True)
print(pretty_ac_json_reg)

{
    "active": false,
    "ageYears": 13.2,
    "airlineName": "Aerolineas Argentinas",
    "deliveryDate": "2011-09-21",
    "engineType": "Jet",
    "firstFlightDate": "2011-09-06",
    "hexIcao": "E06442",
    "iataCodeShort": "738",
    "icaoCode": "B738",
    "id": 14924,
    "isFreighter": false,
    "model": "B738",
    "modelCode": "737-86J",
    "numEngines": 2,
    "numRegistrations": 3,
    "numSeats": 170,
    "productionLine": "Boeing 737 NG",
    "reg": "LV-FQB",
    "registrationDate": "2013-12-20",
    "registrations": [
        {
            "active": true,
            "airlineName": "Avelo Airlines",
            "hexIcao": "AB1A1E",
            "reg": "N814VL",
            "registrationDate": "2024-08-27"
        },
        {
            "active": false,
            "airlineName": "Aerolineas Argentinas",
            "hexIcao": "E06442",
            "reg": "LV-FQB",
            "registrationDate": "2013-12-20"
        },
        {
            "active": false,
       

In [170]:
pretty_ac_json_icao = json.dumps(ac_response_data_icao[0], indent=4, sort_keys=True)
print(pretty_ac_json_icao)

{
    "active": true,
    "airlineName": "Unknown/Private owner",
    "hexIcao": "A447D5",
    "id": 171160,
    "isFreighter": false,
    "numRegistrations": 1,
    "reg": "KAP65",
    "registrations": [
        {
            "active": true,
            "airlineName": "Unknown/Private owner",
            "hexIcao": "A447D5",
            "reg": "KAP65"
        }
    ],
    "typeName": "C402",
    "verified": false
}


Define and extract useful information into a dataframe such that each row represents an aircraft

In [171]:
df_ac_response_raw = pd.DataFrame(ac_response_data_reg)
df_ac_response_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                802 non-null    int64  
 1   reg               802 non-null    object 
 2   active            802 non-null    bool   
 3   serial            771 non-null    object 
 4   hexIcao           772 non-null    object 
 5   airlineName       802 non-null    object 
 6   iataCodeShort     759 non-null    object 
 7   icaoCode          722 non-null    object 
 8   model             761 non-null    object 
 9   modelCode         771 non-null    object 
 10  numSeats          590 non-null    float64
 11  rolloutDate       680 non-null    object 
 12  firstFlightDate   634 non-null    object 
 13  deliveryDate      695 non-null    object 
 14  registrationDate  764 non-null    object 
 15  typeName          802 non-null    object 
 16  numEngines        759 non-null    float64
 1

In [172]:
df_ac_response_raw.rename(columns={"reg" : "registration",
                                   "hexIcao": "icao24"}, inplace=True)
df_ac_response_raw['icao24'] = df_ac_response_raw['icao24'].str.lower()
df_ac_response_raw.head()

,id,registration,active,serial,icao24,airlineName,iataCodeShort,icaoCode,model,modelCode,...,registrationDate,typeName,numEngines,engineType,isFreighter,productionLine,ageYears,verified,numRegistrations,registrations
0,14924,LV-FQB,False,36886,e06442,Aerolineas Argentinas,738,B738,B738,737-86J,...,2013-12-20,Boeing 737-800,2.0,Jet,False,Boeing 737 NG,13.2,True,3,"[{'reg': 'N814VL', 'active': True, 'hexIcao': ..."
1,7594,N183DN,False,27110,a14c29,Delta Air Lines,76W,B763,B763,767-332ER,...,1993-04-29,Boeing 767-300,2.0,Jet,False,Boeing 767,31.6,True,1,"[{'reg': 'N183DN', 'active': False, 'hexIcao':..."
2,15404,N768JB,True,3760,aa600a,JetBlue Airways,320,A320,A320,320-232,...,2009-01-29,Airbus A320,2.0,Jet,False,Airbus A320,15.9,True,1,"[{'reg': 'N768JB', 'active': True, 'hexIcao': ..."
3,27365,N818NW,True,857,ab2855,Delta Air Lines,330,A330,A333,330-323X,...,2009-12-06,Airbus A330,2.0,Jet,False,Airbus A330,17.4,True,2,"[{'reg': 'N818NW', 'active': True, 'hexIcao': ..."
4,22702,HB-JMF,True,561,4b1901,Edelweiss Air,340,A340,A343,340-313X,...,2017-03-14,Airbus A340,4.0,Jet,False,Airbus A340,21.1,True,2,"[{'reg': 'HB-JMF', 'active': True, 'hexIcao': ..."


### Drop Useless Columns and Duplicates

In [173]:
drop_list_ac_response = ['id',                  # identifier particular to the API
                         'active',              # not relevant to study
                         'serial',              # not relevant to study
                         'iataCodeShort',       # we have an ICAO code equivalent and this is redundant
                         'numSeats',            # not relevant to study / highly variable by ac config.
                         'rolloutDate',         # we have an age and this is redundant
                         'firstFlightDate',     # we have an age and this is redundant
                         'deliveryDate',        # we have an age and this is redundant
                         'numEngines',          # not relevant to the study
                         'engineType',          # not relevant to the study (note this just specifies jet, not the engine model)
                         'modelCode',           # not relevant to the study, redundant with production line
                         'typeName',            # not relevant to the study, redundant with production line
                         'numRegistrations',    # may be relevant to study but not including at this time due to processing (what may be relevant is the number of registrations prior to the emergency registration)
                         'registrationDate',    # see 'numRegistrations'
                         'registrations'        # see 'numRegistrations'
                         ]

In [174]:
df_ac_response_raw.drop(columns=drop_list_ac_response, inplace=True)
df_ac_response_raw.drop_duplicates(subset='registration', keep='first',inplace=True)
df_ac_response_raw.head()
print(len(df_ac_response_raw))

767


### Merge with Squawk Dataframe

In [175]:
df_squawk7700_raw = pd.read_csv('../data/raw/squawk7700_metadata.csv')
df_squawk7700_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832 entries, 0 to 831
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   flight_id       832 non-null    object
 1   callsign        832 non-null    object
 2   number          760 non-null    object
 3   icao24          832 non-null    object
 4   registration    832 non-null    object
 5   typecode        832 non-null    object
 6   origin          779 non-null    object
 7   landing         606 non-null    object
 8   destination     773 non-null    object
 9   diverted        295 non-null    object
 10  tweet_problem   419 non-null    object
 11  tweet_result    419 non-null    object
 12  tweet_fueldump  419 non-null    object
 13  avh_id          90 non-null     object
 14  avh_problem     90 non-null     object
 15  avh_result      90 non-null     object
 16  avh_fueldump    90 non-null     object
dtypes: object(17)
memory usage: 110.6+ KB


In [176]:
df_squawk7700_raw.head()

,flight_id,callsign,number,icao24,registration,typecode,origin,landing,destination,diverted,tweet_problem,tweet_result,tweet_fueldump,avh_id,avh_problem,avh_result,avh_fueldump
0,ARG1511_20180101,ARG1511,AR1511,e06442,LV-FQB,B738,SACO,SABE,SABE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAL14_20180101,DAL14,DL14,a14c29,N183DN,B763,KATL,NaN,EDDF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JBU263_20180108,JBU263,B6263,aa600a,N768JB,A320,KJFK,NaN,KSEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DAL65_20180108,DAL65,DL65,ab2855,N818NW,A333,KATL,KLAX,KLAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EDW24_20180111,EDW24,WK24,4b1901,HB-JMF,A343,LSZH,LSZH,MMUN,LSZH,engine,return,unknown,4b382175,engine,return,unknown


In [177]:
drop_list_squawk7700 = ['callsign', # not relevant to study
                        'number'    # not relevant to study
                        ]    
      
df_squawk7700_raw.drop(columns=drop_list_squawk7700, inplace=True)         

In [178]:
df_squawk7700_pre_processed = pd.merge(df_squawk7700_raw, 
                                       df_ac_response_raw, 
                                       on=['registration','icao24'], 
                                       how='left')
df_squawk7700_pre_processed.head()

,flight_id,icao24,registration,typecode,origin,landing,destination,diverted,tweet_problem,tweet_result,...,avh_problem,avh_result,avh_fueldump,airlineName,icaoCode,model,isFreighter,productionLine,ageYears,verified
0,ARG1511_20180101,e06442,LV-FQB,B738,SACO,SABE,SABE,NaN,NaN,NaN,...,NaN,NaN,NaN,Aerolineas Argentinas,B738,B738,False,Boeing 737 NG,13.2,True
1,DAL14_20180101,a14c29,N183DN,B763,KATL,NaN,EDDF,NaN,NaN,NaN,...,NaN,NaN,NaN,Delta Air Lines,B763,B763,False,Boeing 767,31.6,True
2,JBU263_20180108,aa600a,N768JB,A320,KJFK,NaN,KSEA,NaN,NaN,NaN,...,NaN,NaN,NaN,JetBlue Airways,A320,A320,False,Airbus A320,15.9,True
3,DAL65_20180108,ab2855,N818NW,A333,KATL,KLAX,KLAX,NaN,NaN,NaN,...,NaN,NaN,NaN,Delta Air Lines,A330,A333,False,Airbus A330,17.4,True
4,EDW24_20180111,4b1901,HB-JMF,A343,LSZH,LSZH,MMUN,LSZH,engine,return,...,engine,return,unknown,Edelweiss Air,A340,A343,False,Airbus A340,21.1,True


In [179]:
print(len(df_ac_response_raw),
      len(df_squawk7700_raw),
      len(df_squawk7700_pre_processed)
      )

767 832 832


### Additional Processing

In [180]:
mfg_dict = {'Boeing 737 NG' : 'Boeing', 
            'Boeing 767' : 'Boeing',  
            'Airbus A320' : 'Airbus', 
            'Airbus A330' : 'Airbus',
            'Airbus A340' : 'Airbus', 
            'Boeing 737' : 'Boeing',  
            'Boeing 777' : 'Boeing',  
            'Boeing 787' : 'Boeing', 
            'Boeing 757' : 'Boeing',  
            'Airbus A319' : 'Airbus', 
            'Canadair CRJ 900' : 'Canadair',
            'McDonnell Douglas MD-11' : 'McDonnell Douglas',  
            'Airbus A380' : 'Airbus', 
            'Airbus A300' : 'Airbus',
            'Boeing 737 Classic' : 'Boeing',  
            'Airbus A321' : 'Airbus', 
            'Bombardier DHC-8' : 'Bombardier',
            'Airbus A350' : 'Airbus', 
            'Boeing 747' : 'Boeing',  
            'Airbus A220' : 'Airbus', 
            'Canadair CRJ' : 'Canadair',
            'Embraer ERJ170' : 'Embraer',  
            'Canadair CRJ 100' : 'Canadair', 
            'Embraer ERJ145' : 'Embraer', 
            'McDonnell Douglas MD-90' : 'McDonnell Douglas', 
            'McDonnell Douglas MD-88' : 'McDonnell Douglas', 
            'Boeing 737 Original' : 'Boeing',  
            'Airubs A318' : 'Airbus', 
            'Canadair CRJ 700' : 'Canadair',
            'Embraer ERJ190' : 'Embraer',  
            'BAe Avro RJ' : 'BAE', 
            'Airbus A320 NEO' : 'Airbus',
            'Embraer ERJ135' : 'Embraer', 
            'Embraer ERJ140' : 'Embraer',  
            'Fokker 100' : 'Fokker',
            'Boeing 727' : 'Boeing', 
            'ATR 72' : 'ATR', 
            'Sukhoi Superjet 100' : 'Sukhoi', 
            'Boeing 717' : 'Boeing'}

In [181]:
# Drop known medicals
df_squawk7700_processed = df_squawk7700_pre_processed[df_squawk7700_pre_processed['tweet_problem'] != 'medical']
print(f"Dropping known medicals changes size from {len(df_squawk7700_pre_processed)} to {len(df_squawk7700_processed)}")

# Drop verified = False
df_squawk7700_processed = df_squawk7700_processed[df_squawk7700_processed['verified'] != False]
df_squawk7700_processed.drop(columns=['verified'], inplace=True)
print(f"Dropping unverified rows changes size to {len(df_squawk7700_processed)}")

# Drop age = missing
df_squawk7700_processed.dropna(subset=['ageYears'], inplace=True)
print(f"Dropping missing age values changes size to {len(df_squawk7700_processed)}")

# If typecode matches icaocode or model, keep typecode - drop otherwise
# df_squawk7700_processed = df_squawk7700_processed[(df_squawk7700_processed['typecode']==df_squawk7700_processed['icaoCode']) | (df_squawk7700_processed['typecode']==df_squawk7700_processed['model'])]
df_squawk7700_processed.drop(columns=['icaoCode','model'], inplace=True)
# print(f"Dropping unmatched typecodes changes size to {len(df_squawk7700_processed)}")

# Drop productionLine = missing & make Boeing 737 with Boeing 737NG
df_squawk7700_processed.dropna(subset=['productionLine'], inplace=True)
df_squawk7700_processed['productionLine'] = df_squawk7700_processed['productionLine'].replace('Boeing 737', 'Boeing 737 NG')
print(f"Dropping missing productionLine entries changes size to {len(df_squawk7700_processed)}")

# Production Line: extract (no need to drop missing values but worth noting that missing values correspond to smaller AC
df_squawk7700_processed['manufacturer'] = df_squawk7700_processed['productionLine'].map(mfg_dict)
print(f"Mapping manufacturers changes size to {len(df_squawk7700_processed)}")


Dropping known medicals changes size from 832 to 760
Dropping unverified rows changes size to 727
Dropping missing age values changes size to 640
Dropping missing productionLine entries changes size to 635
Mapping manufacturers changes size to 635


In [182]:
# Production List Dictionary
production_dict = {
    'ATR 72': 1233,
    'Airbus A220': 367,
    'Airbus A300': 561,
    'Airbus A319': 1484,
    'Airbus A320': 4752,
    'Airbus A320 NEO': 3607,
    'Airbus A321': 1784,
    'Airbus A330': 1615,
    'Airbus A340': 380,
    'Airbus A350': 623,
    'Airbus A380': 251,
    'Airbus A318': 80,
    'BAe Avro RJ': 387,
    'Boeing 717': 156,
    'Boeing 727': 1832,
    'Boeing 737': 6960,
    'Boeing 737 Classic': 1988,
    'Boeing 737 NG': 6960,
    'Boeing 737 Original': 1150,
    'Boeing 747': 1574,
    'Boeing 757': 1050,
    'Boeing 767': 1319,
    'Boeing 777': 1738,
    'Boeing 787': 1150,
    'Bombardier DHC-8': 1258,
    'Canadair CRJ': 1950,
    'Canadair CRJ 100': 226,
    'Canadair CRJ 700': 330,
    'Canadair CRJ 900': 500,
    'Embraer ERJ135': 120,
    'Embraer ERJ140': 74,
    'Embraer ERJ145': 1100,
    'Embraer ERJ170': 190,
    'Embraer ERJ190': 560,
    'Fokker 100': 283,
    'McDonnell Douglas MD-11': 200,
    'McDonnell Douglas MD-88': 150,
    'McDonnell Douglas MD-90': 116,
    'Sukhoi Superjet 100': 200
}

### Save to Processed

In [183]:
df_squawk7700_processed.to_csv('../data/processed/squawk7700_processed_final_v2.csv', index=False)

with open('../data/processed/production_dict.json', 'w') as production_dict_json_file:
    json.dump(production_dict, production_dict_json_file) 